# Query Antonella's and Y3's tables to find hosts for candidates

In [1]:
import fitsio
import easyaccess as ea
import numpy as np
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
from astropy.coordinates import SkyCoord
import pandas as pd

In [2]:
con = ea.connect('dessci')

Connecting to DB ** dessci ** ...
*Important* Last time password change was 399 days ago
 Please change it using the set_password command to get rid of this message



Load ra, dec, snid, and gcn name for our candidates

In [3]:
ra_cands = [91.526744,93.704382,92.851468,87.311398,89.100926,92.294458,92.468923,87.762354,91.936973,92.307977,87.146843]
dec_cands = [-35.541616,-36.980727,-36.517324,-35.955853,-30.473987,-34.884684,-34.08657,-27.956502,-30.824747,-35.149829,-35.994357]
snids = [479906,480122,479420,479601,479824,480031,480153,479680,479596,479812,480400]
names = ['desgw-190510a','desgw-190510b','desgw-190510c','desgw-190510d','desgw-190510e','desgw-190510f','desgw-190510g','desgw-190510h','desgw-190510i','desgw-190510j','desgw-190510k']

In [4]:
r = 0.028
ra_min = [ra - r for ra in ra_cands]
ra_max = [ra + r for ra in ra_cands]
dec_min = [dec - r for dec in dec_cands]
dec_max = [dec + r for dec in dec_cands]

In [20]: radecrange = 0.01*u.deg

In [21]: radecrange.to(u.arcsec)
Out[21]: <Quantity 36. arcsec>

In [22]: arcsec = radecrange.to(u.arcsec)

In [23]: distance_kpc = (arcsec * d_A).to(u.kpc, u.dimensionless_angles())

In [24]: print distance_kpc
35.1898040359 kpc

> 0.004 deg ~ 15kpc ~ 7DLR (Antonella's limit for matching)

In [ ]:
## ra dec range = +/- 0.004 deg
## no redshift limit
## must match one of antonella's catalog objects

queries_004 = {}
for i in range(len(names)):
    query = ("SELECT a.COADD_OBJECT_ID, a.MAG_ABS_I, a.MAG_ABS_R, d.RA, d.DEC, a.SFR_BEST, d.DNF_ZMEAN_SOF, d.MAG_AUTO_G d "
             "FROM Y3_GOLD_2_2 d, palmese.Y3A2_GALAXIES_STELLAR_MASS a " 
             "WHERE RA < "+str(ra_max[i])+"and RA > "+str(ra_min[i])+" "
             "and DEC < "+str(dec_max[i])+" and DEC > "+str(dec_min[i])+" "
             #"and d.DNF_ZMEAN_SOF < 0.3"
            "and a.COADD_OBJECT_ID = d.COADD_OBJECT_ID;"
            )
    df = con.query_to_pandas(query)
    queries_004[names[i]] = df

good = []
for name, val in queries_004.iteritems():
    if val.empty != True:
        good.append(name)
        print name
        print val
        print
for name in names:
    if name not in good:
        print "NO MATCH: ", name

### check what in y3 gold and NOT antonella's cat

In [11]:
queries_range004 = {}
for i in range(len(names)):
    query = ("SELECT d.COADD_OBJECT_ID, d.RA, d.DEC, d.DNF_ZMEAN_SOF,d.MAG_AUTO_G FROM Y3_GOLD_2_2 d" 
         " WHERE  RA <  "+str(ra_max[i])+" and RA > "+str(ra_min[i])+" and DEC <  "+str(dec_max[i])+" "
         "and DEC >  "+str(dec_min[i])+" "
            "and d.DNF_ZMEAN_SOF < 0.2")
    df = con.query_to_pandas(query)
    queries_range004[names[i]] = df
    #print snids[i]
    #print df

In [ ]:
## ra dec range = +/- 0.001 deg
## z < 0.2
## looking in y3 gold only

nomatch_001 = []
for name, val in queries_range001.iteritems():
    if val.empty != True:
        print name
        print val
        print
    else:
        nomatch_001.append(name)
print "NO MATCH: ", nomatch_001

In [ ]:
## ra dec range = +/- 0.004 deg
## z < 0.2
## looking in y3 gold only

nomatch_004 = []
for name, val in queries_range004.iteritems():
    if val.empty != True:
        print name
        print val
        print
    else:
        nomatch_004.append(name)
print "NO MATCH: ", nomatch_004

In [27]:
for name in nomatch_001:
    if name in nomatch_004:
        print "NO MATCH 004 AND 001", name

NO MATCH 004 AND 001 desgw-190510b
NO MATCH 004 AND 001 desgw-190510c
NO MATCH 004 AND 001 desgw-190510f
NO MATCH 004 AND 001 desgw-190510g


### Antonella limit (100kpc) + z < 0.2, y3 gold only

In [26]:
queries_range100kpc = {}
for i in range(len(names)):
    #SELECT a.COADD_OBJECT_ID, a.MAG_ABS_I, a.MAG_ABS_R, d.RA, d.DEC, a.SFR_BEST, d.DNF_ZMEAN_SOF, d.MAG_AUTO_G d
    query = ("SELECT d.COADD_OBJECT_ID, d.RA, d.DEC, d.DNF_ZMEAN_SOF,d.MAG_AUTO_G FROM Y3_GOLD_2_2 d" 
         " WHERE  RA <  "+str(ra_max[i])+" and RA > "+str(ra_min[i])+" and DEC <  "+str(dec_max[i])+" "
         "and DEC >  "+str(dec_min[i])+" "
            "and d.DNF_ZMEAN_SOF < 0.2")
    df = con.query_to_pandas(query)
    queries_range100kpc[names[i]] = df

In [ ]:
nomatch_100kpc = []
for name, val in queries_range100kpc.iteritems():
    if val.empty != True:
        print name
        print val
        print
    else:
        nomatch_100kpc.append(name)
print "NO MATCH: ", nomatch_100kpc

In [24]:
## NO HOST FOR CANDIDATE B???? 

r = 0.13
ra_min = [ra - r for ra in ra_cands]
ra_max = [ra + r for ra in ra_cands]
dec_min = [dec - r for dec in dec_cands]
dec_max = [dec + r for dec in dec_cands]

i = 1
print names[i]
query = ("SELECT d.COADD_OBJECT_ID, d.RA, d.DEC, d.DNF_ZMEAN_SOF,d.MAG_AUTO_G FROM Y3_GOLD_2_2 d" 
         " WHERE  RA <  "+str(ra_max[i])+" and RA > "+str(ra_min[i])+" and DEC <  "+str(dec_max[i])+" "
         "and DEC >  "+str(dec_min[i])+" ")
          #  "and d.DNF_ZMEAN_SOF < 0.2")
df = con.query_to_pandas(query)
#queries_range100kpc[names[i]] = df
print df

desgw-190510b
    COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0         459527646  93.574651 -37.020820       4.405808   23.925190
1         459524462  93.574683 -36.962244       4.405808   23.181145
2         459526210  93.574763 -36.994809       4.405808   23.743250
3         459527890  93.574770 -37.025495       4.405808   21.948715
4         459532264  93.574902 -37.109816       4.405808   23.166393
5         459523301  93.574990 -36.941601       4.405808   23.333576
6         459531149  93.575011 -37.088431       4.405808   22.295197
7         459527781  93.575023 -37.022904       4.405808   23.736427
8         459523302  93.575039 -36.942496       4.405808   21.560202
9         459525758  93.575100 -36.985952       4.405808   23.391661
10        459521691  93.575266 -36.912498       4.405808   23.625881
11        459525646  93.575304 -36.983979       4.405808   21.778568
12        459530614  93.575344 -37.077712       4.405808   22.869020
13        459527241 

## All ranges [0.001, 0.004, 0.02] + z< 0.2, y3 gold only

In [66]:
#dists = [0.001,0.004, 0.028]
dists = [0.001,0.004, 0.008]
closest_host = {}
done = []
for r in dists:
    ra_min = [ra - r for ra in ra_cands]
    ra_max = [ra + r for ra in ra_cands]
    dec_min = [dec - r for dec in dec_cands]
    dec_max = [dec + r for dec in dec_cands]
    for i in range(len(names)):
        if names[i] in done:
            continue
        else:
            query = ("SELECT d.COADD_OBJECT_ID, d.RA, d.DEC, d.DNF_ZMEAN_SOF,d.MAG_AUTO_G FROM Y3_GOLD_2_2 d" 
                     " WHERE  RA <  "+str(ra_max[i])+" and RA > "+str(ra_min[i])+" and DEC <  "+str(dec_max[i])+" "
                     "and DEC >  "+str(dec_min[i])+" "
                      "and d.DNF_ZMEAN_SOF < 0.2")
            df = con.query_to_pandas(query)
        if df.empty != True:
            closest_host[names[i]] = [r, df]
            done.append(names[i])

In [68]:
#print done, len(done)
for key, val in closest_host.iteritems():
    print key
    print val
    print
    
for i in range(len(names)):
    if names[i] not in done:
        print names[i]
#a, d, k all have matches on NED

desgw-190510a
[0.004,    COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        456401049  91.524818 -35.542911       0.176671   20.416767
1        456400268  91.526228 -35.542993       0.106310   17.170069]

desgw-190510d
[0.001,    COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        453230153  87.311493 -35.956445       0.129511   20.507114]

desgw-190510e
[0.004,    COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        448690201  89.101725 -30.471644       0.163038   19.453684]

desgw-190510f
[0.008,    COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        457383340  92.301126 -34.880852        0.16756   18.732315]

desgw-190510h
[0.004,    COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        454557474  87.761674 -27.955404       0.048781   16.823931]

desgw-190510i
[0.001,    COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        455788150  91.937468 -30.824642  

## 30kpc range, z<0.2 y3gold only

In [12]:
#dists = [0.001,0.004, 0.028]
dists = [0.0085]
has_host30kpc = {}
done = []
for r in dists:
    ra_min = [ra - r for ra in ra_cands]
    ra_max = [ra + r for ra in ra_cands]
    dec_min = [dec - r for dec in dec_cands]
    dec_max = [dec + r for dec in dec_cands]
    for i in range(len(names)):
        if names[i] in done:
            continue
        else:
            query = ("SELECT d.COADD_OBJECT_ID, d.RA, d.DEC, d.DNF_ZMEAN_SOF,d.MAG_AUTO_G FROM Y3_GOLD_2_2 d" 
                     " WHERE  RA <  "+str(ra_max[i])+" and RA > "+str(ra_min[i])+" and DEC <  "+str(dec_max[i])+" "
                     "and DEC >  "+str(dec_min[i])+" "
                      "and d.DNF_ZMEAN_SOF < 0.2")
            df = con.query_to_pandas(query)
        if df.empty != True:
            has_host30kpc[names[i]] = [r, df]
            done.append(names[i])

In [7]:
for key, val in has_host30kpc.iteritems():
    print key
    print val
print len(done)
print 
for i in range(len(names)):
    if names[i] not in done:
        print names[i]

desgw-190510a
[0.0085,    COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        456401049  91.524818 -35.542911       0.176671   20.416767
1        456400268  91.526228 -35.542993       0.106310   17.170069
2        456400658  91.531056 -35.538668       0.110751   20.873541]
desgw-190510d
[0.0085,    COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        453230153  87.311493 -35.956445       0.129511   20.507114]
desgw-190510e
[0.0085,    COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        448690201  89.101725 -30.471644       0.163038   19.453684
1        448690221  89.105216 -30.471248       0.172430   19.878315]
desgw-190510f
[0.0085,    COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        457383340  92.301126 -34.880852        0.16756   18.732315]
desgw-190510h
[0.0085,    COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        454557474  87.761674 -27.955404       0.048781   16.8239

--> cands a, d, k have found hosts from NED

### Remove z limit to find cands c and g

In [5]:
## remove z limit to see if the host is actually here
dists = [0.0085]
for r in dists:
    ra_min = [ra - r for ra in ra_cands]
    ra_max = [ra + r for ra in ra_cands]
    dec_min = [dec - r for dec in dec_cands]
    dec_max = [dec + r for dec in dec_cands]
    i = 2
    print names[i]
    query = ("SELECT d.COADD_OBJECT_ID, d.RA, d.DEC, d.DNF_ZMEAN_SOF,d.MAG_AUTO_G FROM Y3_GOLD_2_2 d" 
                     " WHERE  RA <  "+str(ra_max[i])+" and RA > "+str(ra_min[i])+" and DEC <  "+str(dec_max[i])+" "
                     "and DEC >  "+str(dec_min[i])+" ")
                     # "and d.DNF_ZMEAN_SOF < 0.2")
    df = con.query_to_pandas(query)
print df
seps = {}
for index, row in df.iterrows():
    seps.setdefault(index,[])
    host = SkyCoord(row['RA'], row['DEC'], unit='degree')
    cand = SkyCoord(ra_cands[i], dec_cands[i], unit='degree')
    seps[index].append(cand.separation(host))
print seps

desgw-190510c
    COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0         459792153  92.844051 -36.511630       0.803930   26.492878
1         459792337  92.845133 -36.515647       0.700878   24.329599
2         459792654  92.847015 -36.523329       0.604040   22.062405
3         459792143  92.847693 -36.511273       0.429173   23.612043
4         459792717  92.851442 -36.524333       0.431779   23.026588
5         459792354  92.851901 -36.517915       0.201937   20.480238
6         459792738  92.854899 -36.524663       0.489235   23.496889
7         459792442  92.855610 -36.518210       1.265313   24.201986
8         459792386  92.856915 -36.516736       0.723458   23.053450
9         459792333  92.857271 -36.515578       0.388863   21.819597
10        459792683  92.858060 -36.523785       0.277683   22.421602
11        459792331  92.858633 -36.516250       0.485026   21.338812
12        459792284  92.859650 -36.514546       0.636434   23.913944
13        459792585 

In [12]:
dists = [0.0085]
for r in dists:
    ra_min = [ra - r for ra in ra_cands]
    ra_max = [ra + r for ra in ra_cands]
    dec_min = [dec - r for dec in dec_cands]
    dec_max = [dec + r for dec in dec_cands]
    i = 6
    print names[i], ra_cands[i], dec_cands[i]
    query = ("SELECT d.COADD_OBJECT_ID, d.RA, d.DEC, d.DNF_ZMEAN_SOF,d.MAG_AUTO_G FROM Y3_GOLD_2_2 d" 
                     " WHERE  RA <  "+str(ra_max[i])+" and RA > "+str(ra_min[i])+" and DEC <  "+str(dec_max[i])+" "
                     "and DEC >  "+str(dec_min[i])+" "
                      "and d.DNF_ZMEAN_SOF < 1")
    df = con.query_to_pandas(query)
print df
seps = {}
for index, row in df.iterrows():
    seps.setdefault(index,[])
    host = SkyCoord(row['RA'], row['DEC'], unit='degree')
    cand = SkyCoord(ra_cands[i], dec_cands[i], unit='degree')
    seps[index].append(cand.separation(host))
print seps

desgw-190510g 92.468923 -34.08657
   COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        455492483  92.461412 -34.094592       0.378409   23.130615
1        455492484  92.468660 -34.094640       0.574666   22.976484
2        455492504  92.473743 -34.094924       0.866220   23.661659
{0: [<Angle 0.01015106 deg>], 1: [<Angle 0.00807294 deg>], 2: [<Angle 0.00925867 deg>]}


## Calc seperation between y3gold hosts and cands

In [30]:
candkeys = []
candra = []
canddec = []
for key in has_host30kpc.iterkeys():
    if key in names:
        ind = names.index(key)
        candkeys.append(names[ind])
        candra.append(ra_cands[ind])
        canddec.append(dec_cands[ind])

In [34]:
seps = {}
for key, cra, cdec, val in zip(candkeys, candra, canddec, has_host30kpc.itervalues()):
    seps.setdefault(key,[])
    df = val[1]
    hra = df['RA'].values
    hdec = df['DEC'].values
    for hostr, hostd in zip(hra,hdec):
        host = SkyCoord(hostr, hostd, unit='degree')
        cand = SkyCoord(cra, cdec, unit='degree')
        #seps[key] = cand.separation(host)
        seps[key].append(cand.separation(host))

In [35]:
seps

{'desgw-190510a': [<Angle 0.00203298 deg>,
  <Angle 0.00143959 deg>,
  <Angle 0.00458277 deg>],
 'desgw-190510d': [<Angle 0.00059697 deg>],
 'desgw-190510e': [<Angle 0.0024421 deg>, <Angle 0.00460142 deg>],
 'desgw-190510f': [<Angle 0.00667864 deg>],
 'desgw-190510h': [<Angle 0.00125155 deg>],
 'desgw-190510i': [<Angle 0.01033428 deg>,
  <Angle 0.00595877 deg>,
  <Angle 0.00559076 deg>,
  <Angle 0.00263448 deg>,
  <Angle 0.00043785 deg>,
  <Angle 0.00482206 deg>,
  <Angle 0.00961737 deg>],
 'desgw-190510j': [<Angle 0.00956364 deg>, <Angle 0.00036501 deg>],
 'desgw-190510k': [<Angle 0.00042447 deg>]}

## dist 0.0085 (30kpc) + z< 0.2 + match with antonella catalog

In [12]:
#dists = [0.001,0.004, 0.028]
dists = [0.0085]
has_host30kpcAP = {}
done = []
for r in dists:
    ra_min = [ra - r for ra in ra_cands]
    ra_max = [ra + r for ra in ra_cands]
    dec_min = [dec - r for dec in dec_cands]
    dec_max = [dec + r for dec in dec_cands]
    for i in range(len(names)):
        if names[i] in done:
            continue
        else:
#             query = ("SELECT d.COADD_OBJECT_ID, d.RA, d.DEC, d.DNF_ZMEAN_SOF,d.MAG_AUTO_G FROM Y3_GOLD_2_2 d" 
#                      " WHERE  RA <  "+str(ra_max[i])+" and RA > "+str(ra_min[i])+" and DEC <  "+str(dec_max[i])+" "
#                      "and DEC >  "+str(dec_min[i])+" "
#                       "and d.DNF_ZMEAN_SOF < 0.2")
            query = ("SELECT a.COADD_OBJECT_ID, a.MAG_ABS_I, a.MAG_ABS_R, d.RA, d.DEC, a.SFR_BEST, d.DNF_ZMEAN_SOF, d.DNF_ZSIGMA_SOF, a.MASS_BEST, d.MAG_AUTO_G d "
             "FROM Y3_GOLD_2_2 d, palmese.Y3A2_GALAXIES_STELLAR_MASS a " 
             "WHERE RA < "+str(ra_max[i])+"and RA > "+str(ra_min[i])+" "
             "and DEC < "+str(dec_max[i])+" and DEC > "+str(dec_min[i])+" "
             "and d.DNF_ZMEAN_SOF < 0.2"
            "and a.COADD_OBJECT_ID = d.COADD_OBJECT_ID;"
            )
            df = con.query_to_pandas(query)
    
            if df.empty != True:
                seps = {}
                for index, row in df.iterrows():
                    seps.setdefault(index,[])
                    host = SkyCoord(row['RA'], row['DEC'], unit='degree')
                    cand = SkyCoord(ra_cands[i], dec_cands[i], unit='degree')
                    s = cand.separation(host)
                    seps[index].append(s)

                minindex = [min(seps, key=seps.get)]
                df = df[df.index.isin(minindex)]
                #c = SkyCoord(ra=df['RA'].values*u.degree, dec=df['DEC'].values*u.degree, unit=(u.hourangle))
                print names[i], s.deg
                print df
                df.to_csv("190510_hosts.csv", mode='a', header=False)
                print 
        
        #if df.empty != True:
                has_host30kpcAP[names[i]] = [s, df]
                done.append(names[i])
                


desgw-190510a 0.00458276732965
   COADD_OBJECT_ID  MAG_ABS_I  MAG_ABS_R         RA        DEC  SFR_BEST  DNF_ZMEAN_SOF  DNF_ZSIGMA_SOF  MASS_BEST          D
0        456400268 -23.042999    -22.712  91.526228 -35.542993  -0.18967        0.10631        0.003861    11.2057  17.170069

desgw-190510d 0.000596973623139
   COADD_OBJECT_ID  MAG_ABS_I  MAG_ABS_R         RA        DEC  SFR_BEST  DNF_ZMEAN_SOF  DNF_ZSIGMA_SOF  MASS_BEST          D
0        453230153     -19.33    -19.177  87.311493 -35.956445  -0.50188       0.129511        0.021426    9.43456  20.507114

desgw-190510e 0.00244210274587
   COADD_OBJECT_ID  MAG_ABS_I  MAG_ABS_R         RA        DEC  SFR_BEST  DNF_ZMEAN_SOF  DNF_ZSIGMA_SOF  MASS_BEST          D
0        448690201    -21.796    -21.448  89.101725 -30.471644 -0.389982       0.163038        0.005304    10.7433  19.453684

desgw-190510f 0.00667864112402
   COADD_OBJECT_ID  MAG_ABS_I  MAG_ABS_R         RA        DEC  SFR_BEST  DNF_ZMEAN_SOF  DNF_ZSIGMA_SOF  MASS_BEST  

In [63]:
# for key, val in has_host30kpcAP.iteritems():
#     print key
#     print val
# print len(done)
# print 
# for i in range(len(names)):
#     if names[i] not in done:
#         print names[i]
        
for key, val in has_host30kpcAP.iteritems():
    hz = has_host30kpcAP[key][1]['DNF_ZMEAN_SOF'].values[0]
    cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
    ac = has_host30kpcAP[key][0].to(u.arcsec)
    
    d_A = cosmo.angular_diameter_distance(z=hz)
    distance_kpc = (ac * d_A).to(u.kpc, u.dimensionless_angles())

    print key, val[0].deg, distance_kpc
    print val[1]
    print 
    print

desgw-190510a 0.00458276732965 32.1140385177 kpc
   COADD_OBJECT_ID  MAG_ABS_I  MAG_ABS_R         RA        DEC  SFR_BEST  DNF_ZMEAN_SOF          D
0        456400268 -23.042999    -22.712  91.526228 -35.542993  -0.18967        0.10631  17.170069


desgw-190510d 0.000596973623139 4.96384236215 kpc
   COADD_OBJECT_ID  MAG_ABS_I  MAG_ABS_R         RA        DEC  SFR_BEST  DNF_ZMEAN_SOF          D
0        453230153     -19.33    -19.177  87.311493 -35.956445  -0.50188       0.129511  20.507114


desgw-190510e 0.00244210274587 24.6238343423 kpc
   COADD_OBJECT_ID  MAG_ABS_I  MAG_ABS_R         RA        DEC  SFR_BEST  DNF_ZMEAN_SOF          D
0        448690201    -21.796    -21.448  89.101725 -30.471644 -0.389982       0.163038  19.453684


desgw-190510f 0.00667864112402 68.8639700322 kpc
   COADD_OBJECT_ID  MAG_ABS_I  MAG_ABS_R         RA        DEC  SFR_BEST  DNF_ZMEAN_SOF          D
0        457383340 -22.684999    -22.326  92.301126 -34.880852 -0.155599        0.16756  18.732315


des

## check antonella + y3 list with just y3 list

In [29]:
#has_host30kpc['desgw-190510k'][1]['COADD_OBJECT_ID'].values
for name in names:
    try:
        #if has_host30kpc[name][1]['COADD_OBJECT_ID'].values.any() != has_host30kpcAP[name][1]['COADD_OBJECT_ID'].values.any():
        print name
        print "no ap", has_host30kpc[name][1]['COADD_OBJECT_ID'].values
        print "with ap", has_host30kpcAP[name][1]['COADD_OBJECT_ID'].values
        print
    except KeyError:
        print "n/a"

desgw-190510a
no ap [456401049 456400268 456400658]
with ap [456400268 456400658]

desgw-190510b
no ap n/a
desgw-190510c
no ap n/a
desgw-190510d
no ap [453230153]
with ap [453230153]

desgw-190510e
no ap [448690201 448690221]
with ap [448690201]

desgw-190510f
no ap [457383340]
with ap [457383340]

desgw-190510g
no ap n/a
desgw-190510h
no ap [454557474]
with ap [454557474]

desgw-190510i
no ap [455787889 455788343 455788382 455788145 455788150 455788087 455787940]
with ap [455787889 455788343 455788382 455788150 455788087 455787940]

desgw-190510j
no ap [457397759 457398118]
with ap [457398118]

desgw-190510k
no ap [453232190]
with ap [453232190]



In [46]:
hostra = has_host30kpcAP['desgw-190510i'][1]['RA'].values
hostdec = has_host30kpcAP['desgw-190510i'][1]['DEC'].values
coaddid = has_host30kpcAP['desgw-190510i'][1]['COADD_OBJECT_ID'].values
candra, canddec = 91.936973, -30.824747

seps = []
for hra, hdec, cid in zip(hostra, hostdec, coaddid):
    host = SkyCoord(hra, hdec, unit='degree')
    cand = SkyCoord(candra, canddec, unit='degree')
    seps.append([cid, cand.separation(host)])
print seps

[[455787889, <Angle 0.01033428 deg>], [455788343, <Angle 0.00595877 deg>], [455788382, <Angle 0.00559076 deg>], [455788150, <Angle 0.00043785 deg>], [455788087, <Angle 0.00482206 deg>], [455787940, <Angle 0.00961737 deg>]]


In [16]:
dists = [0.0085]
for r in dists:
    ra_min = [ra - r for ra in ra_cands]
    ra_max = [ra + r for ra in ra_cands]
    dec_min = [dec - r for dec in dec_cands]
    dec_max = [dec + r for dec in dec_cands]
    i = 1
    print names[i], ra_cands[i], dec_cands[i]
    query = ("SELECT d.COADD_OBJECT_ID, d.RA, d.DEC, d.DNF_ZMEAN_SOF,d.MAG_AUTO_G FROM Y3_GOLD_2_2 d" 
                     " WHERE  RA <  "+str(ra_max[i])+" and RA > "+str(ra_min[i])+" and DEC <  "+str(dec_max[i])+" "
                     "and DEC >  "+str(dec_min[i])+" "
                      "and d.DNF_ZMEAN_SOF < 1")
    df = con.query_to_pandas(query)
print df
seps = {}
for index, row in df.iterrows():
    seps.setdefault(index,[])
    host = SkyCoord(row['RA'], row['DEC'], unit='degree')
    cand = SkyCoord(ra_cands[i], dec_cands[i], unit='degree')
    seps[index].append(cand.separation(host))
print seps

desgw-190510b 93.704382 -36.980727
Empty DataFrame
Columns: [COADD_OBJECT_ID, RA, DEC, DNF_ZMEAN_SOF, MAG_AUTO_G]
Index: []
{}


## create DES name for host 
http://cdsweb.u-strasbg.fr/Dic/iau-spec.html#S3.

"Coordinates shall be specified as LLL.ll+BB.bb or LLL.ll–BB.bb for galactic coordinates, and as **HHMMSS.ss+DDMMSS.s or HHMMSS.ss–DDMMSS.s for equatorial coordinates (without spaces)**; more or fewer fractional digits may be used as appropriate, but one should not omit the leading HH or ±DD. Coordinate names expressed in decimal degrees and J2000 should also explicitly include the J"

In [26]:
#c = SkyCoord(ra=92.851901*u.degree, dec=-36.517915*u.degree, unit=(u.hourangle))
bad = ['desgw-190510a', 'desgw-190510d', 'desgw-190510k']
for key, val in has_host30kpcAP.iteritems():
    #print key
    if key not in bad:
        df = val[1]
        if key == 'desgw-190510i':
            value_list = [455788150]
            df = df[df.COADD_OBJECT_ID.isin(value_list)]
        c = SkyCoord(ra=df['RA'].values*u.degree, dec=df['DEC'].values*u.degree, unit=(u.hourangle))
        print key
        print "DES J"+str(c.to_string('hmsdms')[0])
    

desgw-190510e
DES J05h56m24.414s -30d28m17.9184s
desgw-190510f
DES J06h09m12.2702s -34d52m51.0672s
desgw-190510h
DES J05h51m02.8018s -27d57m19.4544s
desgw-190510i
DES J06h07m44.9923s -30d49m28.7112s
desgw-190510j
DES J06h09m14.0189s -35d08m59.0892s


In [14]:
dists = [0.0085]
has_hostCandGkpc = {}
for r in dists:
    ra_min = [ra - r for ra in ra_cands]
    ra_max = [ra + r for ra in ra_cands]
    dec_min = [dec - r for dec in dec_cands]
    dec_max = [dec + r for dec in dec_cands]
#    i = 2 #6
    l = [2,6]
    for i in l: 
        #print names[i]
        query = ("SELECT a.COADD_OBJECT_ID, a.MAG_ABS_I, a.MAG_ABS_R, d.RA, d.DEC, a.SFR_BEST, d.DNF_ZMEAN_SOF,  d.DNF_ZSIGMA_SOF, a.MASS_BEST, d.MAG_AUTO_G d "
                 "FROM Y3_GOLD_2_2 d, palmese.Y3A2_GALAXIES_STELLAR_MASS a " 
             "WHERE RA < "+str(ra_max[i])+"and RA > "+str(ra_min[i])+" "
             "and DEC < "+str(dec_max[i])+" and DEC > "+str(dec_min[i])+" "
             #"and d.DNF_ZMEAN_SOF < 0.2"
            "and a.COADD_OBJECT_ID = d.COADD_OBJECT_ID;"
            )
        df = con.query_to_pandas(query)

        seps = {}
        for index, row in df.iterrows():
            seps.setdefault(index,[])
            host = SkyCoord(row['RA'], row['DEC'], unit='degree')
            cand = SkyCoord(ra_cands[i], dec_cands[i], unit='degree')
            s = cand.separation(host)
            seps[index].append(s)

        minindex = [min(seps, key=seps.get)]
        df = df[df.index.isin(minindex)]
        df.to_csv("190510_hosts.csv", mode='a', header=False)
        #print df
        #print 
        #c = SkyCoord(ra=df['RA'].values*u.degree, dec=df['DEC'].values*u.degree, unit=(u.hourangle))
        #print "DES J"+str(c.to_string('hmsdms')[0])
        #print
        #print names[i], s.deg
        #print df
        #print 
        
        has_hostCandGkpc[names[i]] = [s, df]
#        done.append(names[i])
        
#        query = ("select * from palmese.Y3A2_GALAXIES_STELLAR_MASS where COADD_OBJECT_ID = "+str(df['COADD_OBJECT_ID'].values[0])+";")
#        APdf = con.query_to_pandas(query)
#        print APdf
#         print
#         print
                
for key, val in has_hostCandGkpc.iteritems():
    hz = has_hostCandGkpc[key][1]['DNF_ZMEAN_SOF'].values[0]
    cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
    ac = has_hostCandGkpc[key][0].to(u.arcsec)
    
    d_A = cosmo.angular_diameter_distance(z=hz)
    distance_kpc = (ac * d_A).to(u.kpc, u.dimensionless_angles())

    print key, val[0].deg, distance_kpc
    print val[1]
    print 
    print

desgw-190510c 0.00816273459783 97.6970834039 kpc
   COADD_OBJECT_ID  MAG_ABS_I  MAG_ABS_R         RA        DEC  SFR_BEST  DNF_ZMEAN_SOF  DNF_ZSIGMA_SOF  MASS_BEST          D
2        459792354 -20.070999 -19.959999  92.851901 -36.517915  -0.46222       0.201937         0.09831    9.10912  20.480238


desgw-190510g 0.00925866525825 218.342460769 kpc
   COADD_OBJECT_ID  MAG_ABS_I  MAG_ABS_R        RA       DEC  SFR_BEST  DNF_ZMEAN_SOF  DNF_ZSIGMA_SOF  MASS_BEST          D
1        455492484 -20.556999    -20.476  92.46866 -34.09464  0.259115       0.574666        0.174249    9.43417  22.976484




In [10]:
## cand b
print names[1]
luihdyra = [93.709231, 93.702448, 93.710397, 93.712208, 93.703155, 93.700391, 93.706672]
luihdydec = [-36.979635, -36.984089, -36.984769, -36.982006, -36.977729, -36.979578, -36.983811]
for i in range(len(luihdyra)):
    host = SkyCoord(luihdyra[i], luihdydec[i], unit='degree')
    cand = SkyCoord(ra_cands[1], dec_cands[1], unit='degree')
    sep = cand.separation(host)
    print i, sep.deg

desgw-190510b
0 0.00402457265275
1 0.00369997523829
2 0.00627890582344
3 0.00638114446135
4 0.00315416916658
5 0.00338891307785
6 0.00358572179386


In [21]:
#cand a -- 2MASX J06060625-3532351 (from y3gold: ang sep = 0.00203298)
rahost, dechost = 91.526167, -35.543028
i = 0
host = SkyCoord(rahost, dechost, unit='degree')
cand = SkyCoord(ra_cands[i], dec_cands[i], unit='degree')
sep = cand.separation(host)
print sep.deg
print

#cand d -- MRSS 363-017722 (from y3gold: ang sep = 0.00059697 deg)
rahost, dechost = 87.311745, -35.956402
i = 3
host = SkyCoord(rahost, dechost, unit='degree')
cand = SkyCoord(ra_cands[i], dec_cands[i], unit='degree')
sep = cand.separation(host)
print sep.deg
print

#cand k -- 2MASS J05483537-3559390 (from y3gold: [<Angle 0.00042447 deg>])
rahost, dechost = 87.147510, -35.994156
i = -1
host = SkyCoord(rahost, dechost, unit='degree')
cand = SkyCoord(ra_cands[i], dec_cands[i], unit='degree')
sep = cand.separation(host)
print sep.deg

0.00148800922655

0.000616682559142

0.000575870683128


## gals from NED

In [4]:
ned = pd.read_csv('S190510g_Update_galaxie_fromNED.csv')

In [7]:
#ned.columns
for i in range(len(ra_cands)):
    seps = {}
    for index, row in ned.iterrows():
        seps.setdefault(index,[])
        host = SkyCoord(row['ra'], row['dec'], unit='degree')
        cand = SkyCoord(ra_cands[i], dec_cands[i], unit='degree')
        seps[index].append(cand.separation(host))
    minindex = [min(seps, key=seps.get)]
    print names[i], seps[minindex[0]]
    print ned[ned.index.isin(minindex)]
    print 

 desgw-190510a [<Angle 0.13891505 deg>]
                     objname         ra        dec pretype         z      zunc     DistMpc    DistMod DistMethod    m_Ks  k_msig_k20fe       M_Ks  Merr_Ks  m_FUV  magerr_fuv  M_FUV  Merr_FUV     dP_dV      dP_dA
144  2MASX J06064544-3535032  91.689292 -35.584194       G  0.032923  0.000103  137.087723  35.684994  Kinematic  12.502         0.093 -23.182995    0.093    NaN         NaN    NaN       NaN  0.000008  40.592823

desgw-190510b [<Angle 0.12540741 deg>]
                     objname         ra        dec pretype        z     zunc     DistMpc    DistMod DistMethod    m_Ks  k_msig_k20fe       M_Ks  Merr_Ks  m_FUV  magerr_fuv  M_FUV  Merr_FUV     dP_dV     dP_dA
371  2MASX J06152614-3657295  93.858792 -36.958194       G  0.03345  0.00015  139.282089  35.719479  Kinematic  12.944         0.097 -22.775478    0.097    NaN         NaN    NaN       NaN  0.000002  9.462688

desgw-190510c [<Angle 0.05461629 deg>]
                     objname         r

## New query with no z limit

In [33]:
queries_range001_noz = {}
for i in range(len(names)):
    query = ("SELECT d.COADD_OBJECT_ID, d.RA, d.DEC, d.DNF_ZMEAN_SOF,d.MAG_AUTO_G FROM Y3_GOLD_2_2 d" 
         " WHERE  RA <  "+str(ra_max[i])+" and RA > "+str(ra_min[i])+" and DEC <  "+str(dec_max[i])+" "
         "and DEC >  "+str(dec_min[i])+" ")
#            "and d.DNF_ZMEAN_SOF < 0.2")
    df = con.query_to_pandas(query)
    queries_range001_noz[names[i]] = df
    #print snids[i]
    #print df

In [31]:
## ra dec range = +/- 0.004 deg
## no z limit
## looking in y3 gold only

nomatch_004_noz = []
for name, val in queries_range004_noz.iteritems():
    if val.empty != True:
        print name
        print val
        print
    else:
        nomatch_004_noz.append(name)
print "NO MATCH: ", nomatch_004_noz

desgw-190510a
   COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        456401049  91.524818 -35.542911       0.176671   20.416767
1        456400268  91.526228 -35.542993       0.106310   17.170069

desgw-190510c
   COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        459792354  92.851901 -36.517915       0.201937   20.480238

desgw-190510d
   COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        453230080  87.309402 -35.952791       0.647132   24.940639
1        453230153  87.311493 -35.956445       0.129511   20.507114
2        453230342  87.313750 -35.956818       0.950838   24.646782
3        453230415  87.314023 -35.958139       1.014820   24.228729
4        453230453  87.314840 -35.958720       1.221239   24.921663
5        453230501  87.315060 -35.959566       0.937447   99.000000

desgw-190510e
   COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        448690858  89.099708 -30.476820       0.876798  

## Below is scratch work

In [130]:
range006_good = []
for name, df in queries_range006.iteritems():
    if name not in range001_good:
        if name not in range004_good:
            print name
            print df
            #if df.empty != True:
#            range004_good.append(name)
#print range004_good

desgw-190510b
Empty DataFrame
Columns: [COADD_OBJECT_ID, RA, DEC, DNF_ZMEAN_SOF, MAG_AUTO_G]
Index: []
desgw-190510c
    COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0         457898307  92.802825 -36.521433       0.059485   18.396637
1         457899146  92.803070 -36.534102       0.197813   21.486341
2         459789529  92.811626 -36.468657      -0.057920   14.736027
3         459794317  92.811709 -36.565629       0.056982   23.337381
4         459790091  92.812351 -36.467893      -0.010248   15.419420
5         459793306  92.813970 -36.542838       0.030003   15.834031
6         459793294  92.815969 -36.537963       0.107263   21.400417
7         459791590  92.824706 -36.505492       0.088445   19.403677
8         459792191  92.824883 -36.533897       0.086703   15.740997
9         459792871  92.826592 -36.559914       0.005462   15.324553
10        459790378  92.840130 -36.477889       0.172814   19.461145
11        459793927  92.848212 -36.552926       0.11441

### Save out df to fits file:

In [ ]:
result_file = fitsio.FITS('test_query.fits', 'rw')
result_file.write(DF.to_records(index=False))
result_file.close()

In [153]:
i = 1
print names[i]
query = ("SELECT d.COADD_OBJECT_ID, d.RA, d.DEC, d.DNF_ZMEAN_SOF,d.MAG_AUTO_G FROM Y3_GOLD_2_2 d" 
         " WHERE  RA <  "+str(ra_max[i])+" and RA > "+str(ra_min[i])+" and DEC <  "+str(dec_max[i])+" "
         "and DEC >  "+str(dec_min[i])+"and d.DNF_ZMEAN_SOF < 0.15")
df = con.query_to_pandas(query)
print df ## r = 0.28

desgw-190510b
   COADD_OBJECT_ID         RA        DEC  DNF_ZMEAN_SOF  MAG_AUTO_G
0        459538842  93.425019 -37.228927        0.05218   19.322796


In [31]:
df

,COADD_OBJECT_ID,RA,DEC,DNF_ZMEAN_SOF,MAG_AUTO_G
0,459792354,92.851901,-36.517915,0.201937,20.480238


In [4]:
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
import numpy as np

In [5]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
d_A = cosmo.angular_diameter_distance(z=0.05) #Mpc
#arcsec = 15.0*u.arcsec ## 15 arcsec is the limit for being associated with the host
# arcsec.to(u.deg) = 0.00416667 deg
deg = 0.02845 * u.degree
arcsec = deg.to(u.arcsec)
 
distance_kpc = (arcsec * d_A).to(u.kpc, u.dimensionless_angles())
 
print distance_kpc
d_A

100.114992482 kpc


<Quantity 201.62272531 Mpc>

In [57]:
distance_kpc = 30*u.kpc
#distance_kpc = 0.1*u.Mpc
arcsec = (distance_kpc / d_A).to(u.arcsec, u.dimensionless_angles())
print arcsec
arcmin = arcsec.to(u.arcmin)
print arcmin
deg = arcsec.to(u.degree)
print deg

30.6907079931 arcsec
0.511511799885 arcmin
0.00852519666475 deg


In [51]:
np.rad2deg((0.1/200.))*60
arcmin = 0.5*u.arcmin
d

1.7188733853924698

In [47]:
print arcsec
arcmin = arcsec.to(u.arcmin)
print arcmin
deg = arcsec.to(u.degree)
print deg

arcsec = 30*u.arcsec
print arcsec

arcsec.to(u.deg)

1.0 arcsec
0.0166666666667 arcmin
0.000277777777778 deg
30.0 arcsec


<Quantity 0.00833333 deg>

In [18]:
for i in range(len(names)):
    print names[i], ra_cands[i], dec_cands[i]

 desgw-190510a 91.526744 -35.541616
desgw-190510b 93.704382 -36.980727
desgw-190510c 92.851468 -36.517324
desgw-190510d 87.311398 -35.955853
desgw-190510e 89.100926 -30.473987
desgw-190510f 92.294458 -34.884684
desgw-190510g 92.468923 -34.08657
desgw-190510h 87.762354 -27.956502
desgw-190510i 91.936973 -30.824747
desgw-190510j 92.307977 -35.149829
desgw-190510k 87.146843 -35.994357


In [29]:
print candkeys[2]
print candra[2]
print canddec[2]

desgw-190510e
89.100926
-30.473987


In [4]:
desgw-190510b
desgw-190510c
desgw-190510g

SyntaxError: invalid syntax (<ipython-input-4-a8ab0a6d7b4e>, line 1)

In [14]:
deg = 0.0014395 * u.degree
arcsec = deg.to(u.arcsec)
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
d_A = cosmo.angular_diameter_distance(z=0.05)
distance_kpc = (arcsec * d_A).to(u.kpc, u.dimensionless_angles())
 
print distance_kpc

5.06557229097 kpc


In [15]:
has_host30kpcAP['desgw-190510a'][0].deg
print has_host30kpcAP['desgw-190510a'][1]['DNF_ZMEAN_SOF'].values[0]


0.1063096895813942


In [25]:
hdeg = has_host30kpcAP['desgw-190510a'][0].deg
hz = has_host30kpcAP['desgw-190510a'][1]['DNF_ZMEAN_SOF'].values[0]

arcsec = hdeg.to(u.arcsec)
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

d_A = cosmo.angular_diameter_distance(z=hz)
distance_kpc = (arcsec * d_A).to(u.kpc, u.dimensionless_angles())

AttributeError: 'float' object has no attribute 'to'

In [38]:
has_host30kpcAP['desgw-190510a'][0].arcsec  ##22.914035921612186 kpc
x = cosmo.luminosity_distance(hz) 

In [44]:
distance_kpc = (has_host30kpcAP['desgw-190510a'][0].arcsec * x).to(u.kpc, u.dimensionless_angles())

In [45]:
distance_kpc

<Quantity 4935168.07844066 kpc>

In [ ]:
tan(theta/2) = size/2dist
size = d_A * theta

In [8]:
deg = 0.0085 * u.degree
arcsec = deg.to(u.arcsec)
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
d_A = cosmo.angular_diameter_distance(z=0.05)
distance_kpc = (arcsec * d_A).to(u.kpc, u.dimensionless_angles())
 
print distance_kpc

from astropy.cosmology import z_at_value

z_at_value(cosmo.luminosity_distance, 227*u.Mpc)

29.9113334305 kpc


0.05102209622869852

## Calc P_one

In [ ]:
multiply ligo map by 0.8 * 0.8 to get P_one. 
to get P_one = 0.999... 
what is the number N (how many events identical to this one do we need to get 0.999) that we need to multiply
mulitpy by the prob of it being real (0.5) from https://arxiv.org/pdf/1609.09517.pdf

P_i = SUM(e_spatial(0.8)*e_camera(0.8)*e_detection(robs plot)*e_bns(~.42))
P_one = 1 - prod(1-P_i)


In [2]:
import sys

In [12]:
for n in range(1,100):
#    a = np.array([1,2,3])
#    np.cumprod(a)
    spatial = 0.90 #prob area enclosed
    camera = 0.8 #fraction of the camera that is live
    detection = 0.98
    prob = 0.90 #probabilty of being a real source
    P_i = (spatial * camera * detection * prob)
    p_array = np.full(n, P_i)
    P_one = 1 - np.cumprod(1-p_array)
    print P_one[-1]
    if P_one[-1]>=0.999:
        print(n)
        sys.exit()

0.6350400000000002
0.8668041984000001
0.951388860248064
0.9822588784361335
0.9935252002740512
0.9976369570920177
0.9991375838603028
7


SystemExit: 

In [14]:
93, 98, 99, 99
a = (0.8*0.98*.50*.93)
b = (0.8*0.98*.50*.98)
c = (0.8*0.98*.50*.99)
d = (0.8*0.98*.50*.99)
P_one = 1 - (1-a)*(1-b)*(1-c)*(1-d)
print P_one

0.85346824906


In [73]:
np.full(4, 7)

array([7, 7, 7, 7])

In [80]:
a = np.cumprod([1,2,3])

In [82]:
print a

[1 2 6]


In [84]:
1-(1-p_array)

array([0.263424, 0.263424, 0.263424, 0.263424, 0.263424, 0.263424,
       0.263424, 0.263424, 0.263424])

In [ ]:
sigma_z = sqrt(sigma_gw^2 + sigma_gal^2) mean +/1 90%

In [ ]:
dists = [0.0085]
has_hostY6 = {}
done = []
for r in dists:
    ra_min = [ra - r for ra in ra_cands]
    ra_max = [ra + r for ra in ra_cands]
    dec_min = [dec - r for dec in dec_cands]
    dec_max = [dec + r for dec in dec_cands]
    for i in range(2):
        if names[i] in done:
            continue
        else:
#             query = ("SELECT d.COADD_OBJECT_ID, d.RA, d.DEC, d.DNF_ZMEAN_SOF,d.MAG_AUTO_G FROM Y3_GOLD_2_2 d" 
#                      " WHERE  RA <  "+str(ra_max[i])+" and RA > "+str(ra_min[i])+" and DEC <  "+str(dec_max[i])+" "
#                      "and DEC >  "+str(dec_min[i])+" "
#                       "and d.DNF_ZMEAN_SOF < 0.2")
            query = ("SELECT a.COADD_OBJECT_ID, a.MAG_ABS_I, a.MAG_ABS_R, d.RA, d.DEC, a.SFR_BEST, a.MASS_BEST d "
             "FROM Y6A1_FINALCUT_OBJECT d, palmese.Y3A2_GALAXIES_STELLAR_MASS a " 
             "WHERE RA < "+str(ra_max[i])+"and RA > "+str(ra_min[i])+" "
             "and DEC < "+str(dec_max[i])+" and DEC > "+str(dec_min[i])+" "
             "and a.DNF_ZMEAN_SOF < 0.2"
            #"and a.COADD_OBJECT_ID = d.COADD_OBJECT_ID;"
            )
            df = con.query_to_pandas(query)
    
            if df.empty != True:
                seps = {}
                for index, row in df.iterrows():
                    seps.setdefault(index,[])
                    host = SkyCoord(row['RA'], row['DEC'], unit='degree')
                    cand = SkyCoord(ra_cands[i], dec_cands[i], unit='degree')
                    s = cand.separation(host)
                    seps[index].append(s)

                minindex = [min(seps, key=seps.get)]
                df = df[df.index.isin(minindex)]
                #c = SkyCoord(ra=df['RA'].values*u.degree, dec=df['DEC'].values*u.degree, unit=(u.hourangle))
                print names[i], s.deg
                print df
                print 
        
        #if df.empty != True:
                has_hostY6[names[i]] = [s, df]
                done.append(names[i])

In [5]:
query = "select * from Y6A1_FINALCUT_OBJECT@dessci where HPIX_4096 = 83957674 ;"
df = con.query_to_pandas(query)

In [4]:
#df

In [21]:
c = SkyCoord('86.33527', '-26.84768', unit=(u.deg))

In [26]:
c*u.hourangle

TypeError: unsupported operand type(s) for *: 'SkyCoord' and 'Unit'

In [29]:
a = (86.33527, -26.84768)*u.deg

In [30]:
a.to(u.hourangle)

<Quantity [ 5.75568467, -1.78984533] hourangle>

In [9]:
host = SkyCoord(93.704417, -36.980711, unit='degree')
cand = SkyCoord(93.704382, -36.980727, unit='degree')
cand.separation(host)

<Angle 3.22137254e-05 deg>

0d00m00.4458s
